In [8]:
include("NTKernel.jl");

In [9]:
# PSUDEO data
g = x -> cos(2x)+sin(x/2)           # anonymous function g(x)

X = range(1, stop=12, length=100)
X = Matrix(reshape(X, 1, :))        # reshape, each column vector is read as data point

Y = map(g,X);                       # Map with g(x)
Ý = map_model(models[1],X);         # evaluates  the model on X and returns Ý, accounts for dim sizes 😍

In [12]:
# nodes on hidden layer
Nh = 100
InputDim = 1
activation = sigmoid

model = Chain(Dense(InputDim=>Nh,activation),Dense(Nh=>InputDim))|>f64

K = kernel(model, X)
normalized_K = K / sqrt(Nh) # Normalize the kernel matrix

λ_values = eigen(normalized_K).values;

In [ ]:
using Plots

# Create subplots
plot1 = scatter(X[:], Y[:], label="Real data", xlabel="X", ylabel="Y")
scatter!(X[:], Ý[:], label="Estimated data")

plot2 = scatter(λ_values, label="Eigenvalue", xlabel = "Id. Eigenvalue")

# Combine subplots into a single plot
plot(plot1, plot2)

<h1> Now, lets start tinkering with the models </h1>

In [ ]:
# nodes on hidden layer
Nh_list = [100] 
models = []
activation = sigmoid

InputDim = 1

for Nh in 1:length(Nh_list)
    push!(models,Chain(Dense(InputDim=>Nh_list[Nh],activation),Dense(Nh_list[Nh]=>InputDim))|>f64)
end

Kernels = []
for model in models
    push!(Kernels, kernel(model, X))
end

Kernels_Norms = []
for (i, K) in enumerate(Kernels)
    Nh = Nh_list[i] # Get the corresponding Nh for this kernel
    normalized_K = K / sqrt(Nh) # Normalize the kernel matrix
    push!(Kernels_Norms, normalized_K) # Store the normalized kernel
end

loss(a, b) = Flux.Losses.mse(models[1](a), b)
#Flux.train!(loss, Flux.params(models[1]), [(X,Y)], Descent(0.01))

λ_values = []
for K in Kernels_Norms
    push!(λ_values, eigen(K).values) # Compute eigenvalues for each kernel and store
end

λ_values